In [120]:
import pandas as pd
import re

In [122]:

df = pd.read_csv("ai_tweets.csv", encoding='ISO-8859-1')
df.head()

,Tweet,created_at
0,@cryptogems555 Invest in a currency that is de...,3/14/2025
1,While having all the hope around chatGPT lets ...,3/14/2025
2,@IGDEFI1 Join the movement towards a more sust...,3/14/2025
3,"There's a snowstorm out so naturally, I decide...",3/14/2025
4,@unclebobcrypto @Stablzone Invest in a currenc...,3/14/2025


In [123]:
df = df[~df['created_at'].astype(str).str.contains("Time", na=False)]
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df = df[df['created_at'].notna()]

df.drop_duplicates(inplace=True)

df.dropna(subset=['Tweet', 'created_at'], inplace=True)

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+", "", text)  
    text = re.sub(r"@\w+", "", text)    
    text = re.sub(r"#\w+", "", text)     
    text = re.sub(r"[^\w\s]", "", text)  
    text = re.sub(r"\s+", " ", text)     
    return text.strip().lower()

df['cleaned_text'] = df['Tweet'].apply(clean_text)
df.drop_duplicates(subset=['cleaned_text'], inplace=True)

df['Tweet'] = df['cleaned_text']
df.drop(columns=['cleaned_text'], inplace=True)

df.head()


,Tweet,created_at
0,invest in a currency that is designed to offer...,2025-03-14
1,while having all the hope around chatgpt lets ...,2025-03-14
2,join the movement towards a more sustainable a...,2025-03-14
3,theres a snowstorm out so naturally i decided ...,2025-03-14
4,invest in a currency that offers a more secure...,2025-03-14


In [124]:
df.shape

(123746, 2)

In [125]:
na_counts = df.isna().sum()
print(na_counts)


Tweet         0
created_at    0
dtype: int64


In [126]:
duplicate_counts = df.duplicated().sum()
duplicate_counts

0

In [128]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')  

db = client['ai_tweets_db']  
collection = db['tweets']  

data_dict = df.to_dict("records")

if collection.count_documents({}) == 0:
    collection.insert_many(data_dict)
    print("The data has been stored in MongoDB.")
else:
    print("Data already exists in MongoDB. Skipping insert.")


Data already exists in MongoDB. Skipping insert.


In [138]:
count = collection.count_documents({})
print(f"num. of docs.: {count}")


num. of docs.: 123746


In [140]:
for doc in collection.find().limit(5):
    print(doc)


{'_id': ObjectId('680a9c3d4f657be066b8aed9'), 'Tweet': 'invest in a currency that is designed to offer greater flexibility and customization for personal financial strategies and goals with this coin', 'created_at': datetime.datetime(2025, 3, 14, 0, 0)}
{'_id': ObjectId('680a9c3d4f657be066b8aeda'), 'Tweet': 'while having all the hope around chatgpt lets not forget these tools in other domains too', 'created_at': datetime.datetime(2025, 3, 14, 0, 0)}
{'_id': ObjectId('680a9c3d4f657be066b8aedb'), 'Tweet': 'join the movement towards a more sustainable and responsible financial system that prioritizes longterm growth and development 0x73706a7d4c34b3c70a1cd35030b847a0e11403e0', 'created_at': datetime.datetime(2025, 3, 14, 0, 0)}
{'_id': ObjectId('680a9c3d4f657be066b8aedc'), 'Tweet': 'theres a snowstorm out so naturally i decided to ask about ð', 'created_at': datetime.datetime(2025, 3, 14, 0, 0)}
{'_id': ObjectId('680a9c3d4f657be066b8aedd'), 'Tweet': 'invest in a currency that offers a more